In [ ]:
# only Sheet that is  Updating for Nepali Cars sheet
# updated and splitted title column to brandname, year and model
import pandas as pd

# df = pd.read_csv('output_data_final.csv');
sheet_name_or_index = 1
df = pd.read_excel('v1-output-data-updated-column-nepalicars.xlsx',sheet_name='Nepali Cars Data')
brandName = []
for index, eachRow in df.iterrows():
    title=eachRow['title']
    brandName=(title.split()[1])
    year=(title.split()[0])
    model = ' '.join(title.split()[2:4]) if len(title.split()) > 2 else ''
    df.at[index, 'brandName'] = brandName
    df.at[index, 'Make Year'] = year
    df.at[index, 'Model'] = model
    df.to_excel('update_nepali_cars_column.xlsx', index=False)

In [ ]:
#To Copy all the remaining sheets along with the data to update for nepali cars
# updated and splitted title column to brandname, year and model
#  Also copied all the primary sheets remaining

import pandas as pd
from xlsxwriter import Workbook

dfs = pd.read_excel('output_data_final.xlsx', sheet_name=None)

# Specify the sheet you want to update
updating_sheet = 'Nepali Cars Data'

# Extract the DataFrame for the specified sheet
df = dfs[updating_sheet]
brandName = []
for index, eachRow in df.iterrows():
    title = eachRow['title']
    brandName = title.split()[1]
    year = title.split()[0]
    model = ' '.join(title.split()[2:4]) if len(title.split()) > 2 else ''
    df.at[index, 'brandName'] = brandName
    df.at[index, 'Make Year'] = year
    df.at[index, 'Model'] = model

# Save the updated DataFrame back to the Excel file
with pd.ExcelWriter('v1-output-data-updated-column-nepalicars.xlsx', engine='xlsxwriter') as writer:
    for sheet_name, df_sheet in dfs.items():
        if sheet_name == updating_sheet:
            # Save the updated DataFrame for the specified sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        else:
            # Save the unchanged DataFrames for other sheets
            df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

In [47]:
#To Copy all the remaining sheets along with the data to update for Hamrobazar cars
# updated and splitted Transmission column to Drive(4WD, 2Wd), Gear(automatic/manual)
#  Also copied all the primary sheets remaining
import re
import pandas as pd
from xlsxwriter import Workbook

dfs = pd.read_excel('v1-output-data-updated-column-nepalicars.xlsx', sheet_name=None)

# Specify the sheet you want to update
updating_sheet = 'Hamro bazar data'

# Extract the DataFrame for the specified sheet
df = dfs[updating_sheet].copy() 
brandName = []
# for index, eachRow in df.iterrows():

drive_pattern = re.compile(r'(2WD|4WD|4 WD|2 WD)', flags=re.IGNORECASE)
df['Drive'] = (df['Transmission'].apply(lambda x: drive_pattern.search(str(x)) and drive_pattern.search(str(x)).group(0))).replace(" ", "")
# df['Drive'] = df['Transmission'].apply(lambda x: '2WD' if pd.notna(x) and ('2WD' in str(x).upper() or '2 WD' in str(x).upper()) else '4WD' if pd.notna(x) and ('4WD' in str(x).upper() or '4 WD' in str(x).upper()) else None)
automatic_pattern = re.compile(r'(Auto|Automatic|Power\s*Steering|PowerSteering)', flags=re.IGNORECASE)
manual_pattern = re.compile(r'Manual', flags=re.IGNORECASE)

# Apply regex patterns to create the 'Gear' column
df['Gear']  = df['Transmission'].apply(lambda x: 'Automatic' if pd.notna(x) and automatic_pattern.search(str(x)) else 'Manual' if pd.notna(x) and manual_pattern.search(str(x)) else '-')

# df['Gear'] = df['Transmission'].apply(lambda x: 'Automatic' if pd.notna(x) and 'Auto' in str(x) or 'Power Steering' in str(x) else 'Manual')

# df.at[index, 'Drive'] = df['Drive'].fillna('-') 
display(df[['Transmission', 'Gear', 'Drive']])




#Save the updated DataFrame back to the Excel file
with pd.ExcelWriter('v2-data-Transmission-drive-updated-column-hamrobazar.xlsx', engine='xlsxwriter') as writer:
    for sheet_name, df_sheet in dfs.items():
        if sheet_name == updating_sheet:
            # Save the updated DataFrame for the specified sheet
            df_combined = pd.concat([df_sheet, df[[ 'Gear', 'Drive']]], axis=1)
            df_combined.to_excel(writer, sheet_name=sheet_name, index=False)

        else:
            # Save the unchanged DataFrames for other sheets
            df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)



,Transmission,Gear,Drive
0,Auto - 2WD,Automatic,2WD
1,Auto - 2WD,Automatic,2WD
2,Manual2WD,Manual,2WD
3,Manual2WD,Manual,2WD
4,Manual4WD,Manual,4WD
...,...,...,...
3295,Manual Gear - 2 WD,Manual,2 WD
3296,Manual Gear - 2 WD,Manual,2 WD
3297,NaN,-,None
3298,Manual Gear - 2 WD,Manual,2 WD
